In [7]:
import cv2
import numpy as np
import os

In [8]:
video_path = 'data/climate.mp4'
output_folder = 'output/'

if os.path.isdir(output_folder):
    print("Delete old result folder: {}".format(output_folder))
    os.system("rm -rf {}".format(output_folder))
os.system("mkdir {}".format(output_folder))
print("create folder: {}".format(output_folder))

vc = cv2.VideoCapture(video_path)
fps = vc.get(cv2.CAP_PROP_FPS)
frame_count = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_count)
video = []


Delete old result folder: output/
create folder: output/
1780


In [9]:
t = frame_count
digit = 0
while t > 0:
    t = int(t / 10)
    digit += 1
print(digit)

4


In [10]:
hist_base = []
prev_hist = []
frame_flag = np.zeros(frame_count)
frame_gap = 1
vc.set(1, 0)
#frame_count = 1059
for idx in range(0, frame_count, 1):
    
    ret, frame = vc.read()
    height, width, layers = frame.shape
    size = (width, height)

    if frame is not None:
  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
       

        t = []
        for i in range(4):
            for j in range(4):
                h = int(size[1]/4)
                w = int(size[0]/4)
                t.append(cv2.calcHist([gray[h*i:h*(i+1)-1, w*j:w*(j+1)-1]], [0], None, [256], [0, 256]))
                cv2.normalize(t[i*4+j], t[i*4+j], alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        
        hist_base.append(t)
        

        
        if idx >= frame_gap:
            for compare_method in range(1):
                base_base = []
                gap = 15
                for i in range(16):
                    t = cv2.compareHist(hist_base[idx][i], hist_base[idx - frame_gap][i], compare_method)
                    base_base.append(t)

                if len(base_base) > 0:
                   
                    s = ''
                    count = 0
                    for i in range(len(base_base)):
                        if base_base[i] < 0.7:
                            count += 1
                            s += ' {:.4f}'.format(base_base[i])
                        else:
                            s += '       '
                    if count > 8:
                        frame_flag[idx] = 1
                    elif count > 6:
                        frame_flag[idx] = 2
                    elif count > 4:
                        frame_flag[idx] = 3
                    #print('{:4}'.format(idx), ' Method:', compare_method, ' ', s, '/')
    else:
        print('error')

In [11]:

        
s = ''
frame_is_count = np.zeros(frame_count)
for i in range(1, len(frame_flag),frame_gap):
    if frame_flag[i] == 1:
        frame_is_count[i] = 1        
    if frame_flag[i] == 2:
        flag = True
        for j in range(1, 3):
            if i - j < 1:
                break
            if frame_flag[i - j] > 2 or frame_flag[i - j] == 0:
                flag = False
                break
        if flag:
            frame_is_count[i] = 1
    if frame_flag[i] == 3:
        flag = True
        for j in range(1, 6):
            if i - j < 1:
                break
            if frame_flag[i - j] == 0:
                flag = False
                break
        if flag:
            frame_is_count[i] = 1
for i in range(0, len(frame_flag),frame_gap):
    if (i + 1) == len(frame_flag):
        flag = 0
    else:
        flag = frame_is_count[i + 1]
    if frame_is_count[i] == 1:
        if frame_is_count[i - 1] != 1 and flag == 1:
            print(str(i)+'~',end = '')
        elif flag != 1:
            print(i)

92
156
231
313
354
455~465
898~899
914
1020
1236
1400
1554
